In [1]:
import h5py
import json
import numpy as np
import os
import sys
from matplotlib import pyplot as plt

rootFolder = '/home/fernando/connectome_S1_bbp/'
os.chdir(rootFolder)

savedata = 1 # Save 
plotdata = 1 # plot 

mtype_map = {}
Epops = ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
            'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
            'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']
# order from Fig 3 - Cerebral Cortex, September 2017;27: 4570-4585 - doi: 10.1093/cercor/bhx150
mtype_map[1] = 'L1_DAC'
mtype_map[2] = 'L1_NGC-DA'
mtype_map[3] = 'L1_NGC-SA'
mtype_map[4] = 'L1_HAC'
mtype_map[5] = 'L1_DLAC'
mtype_map[6] = 'L1_SLAC'
mtype_map[7] = 'L23_PC'
mtype_map[8] = 'L23_MC'
mtype_map[9] = 'L23_BTC'
mtype_map[10] = 'L23_DBC'
mtype_map[11] = 'L23_BP'
mtype_map[12] = 'L23_NGC'
mtype_map[13] = 'L23_LBC'
mtype_map[14] = 'L23_NBC'
mtype_map[15] = 'L23_SBC'
mtype_map[16] = 'L23_ChC'
mtype_map[17] = 'L4_PC'
mtype_map[18] = 'L4_SP'
mtype_map[19] = 'L4_SS'
mtype_map[20] = 'L4_MC'
mtype_map[21] = 'L4_BTC'
mtype_map[22] = 'L4_DBC'
mtype_map[23] = 'L4_BP'
mtype_map[24] = 'L4_NGC'
mtype_map[25] = 'L4_LBC'
mtype_map[26] = 'L4_NBC'
mtype_map[27] = 'L4_SBC'
mtype_map[28] = 'L4_ChC'
mtype_map[29] = 'L5_TTPC1'
mtype_map[30] = 'L5_TTPC2'
mtype_map[31] = 'L5_UTPC'
mtype_map[32] = 'L5_STPC'
mtype_map[33] = 'L5_MC'
mtype_map[34] = 'L5_BTC'
mtype_map[35] = 'L5_DBC'
mtype_map[36] = 'L5_BP'
mtype_map[37] = 'L5_NGC'
mtype_map[38] = 'L5_LBC'
mtype_map[39] = 'L5_NBC'
mtype_map[40] = 'L5_SBC'
mtype_map[41] = 'L5_ChC'
mtype_map[42] = 'L6_TPC_L1'
mtype_map[43] = 'L6_TPC_L4'
mtype_map[44] = 'L6_UTPC'
mtype_map[45] = 'L6_IPC'
mtype_map[46] = 'L6_BPC'
mtype_map[47] = 'L6_MC'
mtype_map[48] = 'L6_BTC'
mtype_map[49] = 'L6_DBC'
mtype_map[50] = 'L6_BP'
mtype_map[51] = 'L6_NGC'
mtype_map[52] = 'L6_LBC'
mtype_map[53] = 'L6_NBC'
mtype_map[54] = 'L6_SBC'
mtype_map[55] = 'L6_ChC'

from scipy.optimize import curve_fit

Netinfo = {}

def exponential(x, a, b):
    return a*np.exp(-b*x)

def linear(x, a, b):
    return a*x + b

def Gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

data = {}
data['BBP_S1_mc0'] = {}
data['BBP_S1_mc1'] = {}
data['BBP_S1_mc2'] = {}
data['BBP_S1_mc3'] = {}
data['BBP_S1_mc4'] = {}
data['BBP_S1_mc5'] = {}
data['BBP_S1_mc6'] = {}

with open('Netconnections_mc0.json', 'r') as f:
    data['BBP_S1_mc0']['connProb'] = json.load(f) 

with open('Netconnections_mc1.json', 'r') as f:
    data['BBP_S1_mc1']['connProb'] = json.load(f) 

with open('Netconnections_mc2.json', 'r') as f:
    data['BBP_S1_mc2']['connProb'] = json.load(f) 
    
with open('Netconnections_mc3.json', 'r') as f:
    data['BBP_S1_mc3']['connProb'] = json.load(f) 

with open('Netconnections_mc4.json', 'r') as f:
    data['BBP_S1_mc4']['connProb'] = json.load(f) 

with open('Netconnections_mc5.json', 'r') as f:
    data['BBP_S1_mc5']['connProb'] = json.load(f) 
    
with open('Netconnections_mc6.json', 'r') as f:
    data['BBP_S1_mc6']['connProb'] = json.load(f) 

In [2]:
for n in range(1,56):
    for m in range(1,56):

        prob2D = []
        d2D = []
        prob2Dfulldata = []
        d2Dfulldata = []
        expfit = 0
        nn = 0    
        m0 = 0      
        m1 = 0      
        m2 = 0
        m3 = 0    
        m4 = 0
        m5 = 0    
        m6 = 0
        m7 = 0    
        m8 = 0
        m9 = 0   
        m10 = 0
        m11 = 0    
        m12 = 0
        m13 = 0   
        m14 = 0   
        pmat = 0   
        pmat0 = 0   
        mm = 0   
        for mc in range(7):
            pre = mtype_map[n]
            post = mtype_map[m]
            proj = '%s:%s' % (mtype_map[n],mtype_map[m])
            mcName = 'BBP_S1_mc' + str(mc)

            if proj in data[mcName]['connProb']:
            
                pmat = pmat + float(data[mcName]['connProb'][proj]['connection_probability_full'])
                nn =  nn + int(data[mcName]['connProb'][proj]['connections_total'])
                pmat0 = pmat0 + float(data[mcName]['connProb'][proj]['connection_probability_less_25um'])

                m0 = m0 + float(data[mcName]['connProb'][proj]['connection_probability_25um'])
                m1 = m1 + float(data[mcName]['connProb'][proj]['connection_probability_50um'])
                m2 = m2 + float(data[mcName]['connProb'][proj]['connection_probability_75um']) 
                m3 = m3 + float(data[mcName]['connProb'][proj]['connection_probability_100um'])
                m4 = m4 + float(data[mcName]['connProb'][proj]['connection_probability_125um'])
                m5 = m5 + float(data[mcName]['connProb'][proj]['connection_probability_150um'])
                m6 = m6 + float(data[mcName]['connProb'][proj]['connection_probability_175um'])
                m7 = m7 + float(data[mcName]['connProb'][proj]['connection_probability_200um'])
                m8 = m8 + float(data[mcName]['connProb'][proj]['connection_probability_225um'])
                m9 = m9 + float(data[mcName]['connProb'][proj]['connection_probability_250um'])
                m10 = m10 + float(data[mcName]['connProb'][proj]['connection_probability_275um'])
                m11 = m11 + float(data[mcName]['connProb'][proj]['connection_probability_300um'])
                m12 = m12 + float(data[mcName]['connProb'][proj]['connection_probability_325um'])
                m13 = m13 + float(data[mcName]['connProb'][proj]['connection_probability_350um'])
                m14 = m14 + float(data[mcName]['connProb'][proj]['connection_probability_375um'])

                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_less_25um']))   
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_25um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_50um']))   
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_75um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_100um']))   
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_125um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_150um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_175um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_200um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_225um']))      
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_250um']))   
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_275um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_300um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_325um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_350um']))    
                prob2Dfulldata.append(float(data[mcName]['connProb'][proj]['connection_probability_375um']))   
                            
                d2Dfulldata.append(12.5)
                d2Dfulldata.append(25)
                d2Dfulldata.append(50) 
                d2Dfulldata.append(75)     
                d2Dfulldata.append(100)
                d2Dfulldata.append(125)
                d2Dfulldata.append(150)
                d2Dfulldata.append(175)    
                d2Dfulldata.append(200)      
                d2Dfulldata.append(225)  
                d2Dfulldata.append(250)
                d2Dfulldata.append(275)
                d2Dfulldata.append(300)
                d2Dfulldata.append(325)    
                d2Dfulldata.append(350)      
                d2Dfulldata.append(375)

                mm = mm + 1

        if mm == 7:
            prob2D.append(pmat0/mm)   
            prob2D.append(m0/mm)       
            prob2D.append(m1/mm)      
            prob2D.append(m2/mm)      
            prob2D.append(m3/mm)      
            prob2D.append(m4/mm)      
            prob2D.append(m5/mm)       
            prob2D.append(m6/mm)      
            prob2D.append(m7/mm)      
            prob2D.append(m8/mm)          
            prob2D.append(m9/mm)      
            prob2D.append(m10/mm)      
            prob2D.append(m11/mm)       
            prob2D.append(m12/mm)      
            prob2D.append(m13/mm)      
            prob2D.append(m14/mm)    
                        
            d2D.append(12.5)
            d2D.append(25)
            d2D.append(50) 
            d2D.append(75)     
            d2D.append(100)
            d2D.append(125)
            d2D.append(150)
            d2D.append(175)    
            d2D.append(200)      
            d2D.append(225)
            d2D.append(250)
            d2D.append(275)
            d2D.append(300)
            d2D.append(325)    
            d2D.append(350)      
            d2D.append(375)

            x = d2D
            y = prob2D
            
            if max(y) == y[0]: # 25 um 
                starts = 0
            elif max(y[1:]) == y[1]:
                starts = 1 # 50 um 
            elif max(y[2:]) == y[2]:
                starts = 2 # 75 um 
            elif max(y[3:]) == y[3]:
                starts = 3 # 100 um 
            elif max(y[4:]) == y[4]:
                starts = 4 # 125 um 
            elif max(y[5:]) == y[5]:
                starts = 5 # 150 um 
            elif max(y[6:]) == y[6]:
                starts = 6 # 175 um 
            else:
                starts = 7 # 200 um 

            ends = 16
            if y[15] == 0: # 375 um 
                ends = ends - 1
                if y[14] == 0: # 375 um 
                    ends = ends - 1
                    if y[13] == 0: # 350 um 
                        ends = ends - 1
                        if y[12] == 0: # 325 um 
                            ends = ends - 1
                            if y[11] == 0: # 300 um 
                                ends = ends - 1
                                if y[10] == 0: # 275 um 
                                    ends = ends - 1
                                    if y[9] == 0: # 250 um 
                                        ends = ends - 1
                                        if y[8] == 0: # 225 um 
                                            ends = ends - 1
                                            if y[7] == 0: # 200 um 
                                                ends = ends - 1
                                                if y[6] == 0: # 175 um 
                                                    ends = ends - 1
                                                    if y[5] == 0: # 150 um 
                                                        ends = ends - 1
                                                        if y[4] == 0: # 125 um 
                                                            ends = ends - 1

            x = d2D[starts:ends]
            y = prob2D[starts:ends]

            if x[-1] == 375:
                endsexp = 400
            else:
                endsexp = x[-1]

            #saturation correction
            if x[0] < 100 and (ends-starts) > 9:

                if x[0] == 12.5: # tolerance
                    fracx0 = 0.525
                else:
                    fracx0 = 1.05

                if (y[4]-y[5]) < 1.05*(y[5]-y[6]):
                    starts = starts + 5 
                elif (y[3]-y[4]) < 1.05*(y[4]-y[5]):
                    starts = starts + 4 
                elif (y[2]-y[3]) < 1.05*(y[3]-y[4]):
                    starts = starts + 3 
                elif (y[1]-y[2]) < 1.05*(y[2]-y[3]):
                    starts = starts + 2 
                elif (y[0]-y[1]) < fracx0*(y[1]-y[2]):
                    starts = starts + 1 

            # if n == 1 and m == 11: #ajusts by hand 
            #     starts = 2
            # if n == 1 and m == 19: #ajusts by hand
            #     starts = 4

            x = d2D[starts:ends]
            y = prob2D[starts:ends]

            # print(x,y)
            pars, cov = curve_fit(f=exponential, xdata=x, ydata=y, p0=[0, 0], bounds=(-np.inf, np.inf))

            if x[0] == 12.5:
                xx = np.linspace(0, endsexp, 50)
                yy = exponential(xx, *pars)
                xi = -1
                yi = 0
            elif x[0] == 25:
                xx = np.linspace(25, endsexp, 50)
                yy = exponential(xx, *pars)
                xi = [0, 25]
                yi = [prob2D[0], prob2D[0]] 
            else:
                xx = np.linspace(x[0], endsexp, 50)
                yy = exponential(xx, *pars)                    
                y1 = prob2D[1]
                x1 = 25
                y2 = y[0]
                x2 = x[0] 
                a = (y2 - y1)/(x2 - x1)
                b = y2 - x2*a    
                xi = np.linspace(0, x[0], 11)
                yi = linear(xi,a, b)

            cov1 = 1000000*cov[0,0]

            
            xg = np.array(d2D[1:ends])
            yg = np.array(prob2D[1:ends])
            # weighted arithmetic mean 
            meang = sum(xg * yg) / sum(yg)
            sigmag = np.sqrt(sum(yg * (xg - meang)**2) / sum(yg))                
            popt = [0, 0, 1]            
            pcov = np.zeros((3,3))
            try:  
                popt,pcov = curve_fit(Gauss, xg, yg, p0=[max(yg), meang, sigmag])
            except:
                Optimalparameters = 'not found'
#                 print('Optimal parameters not found for %s' % (proj))    
                
            cov2 = 1000000*pcov[0,0]
                        
            
            proj = '%s:%s' % (mtype_map[n],mtype_map[m])
            Netinfo[proj] = {}
            Netinfo[proj]['conn_total'] = '%.1f' % (nn/mm) 
            Netinfo[proj]['conn_prob_full'] = '%.5f' % (pmat/mm) 
            Netinfo[proj]['A0'] = '%.5f' % pars[0]
            Netinfo[proj]['shape'] = '%.2f' % (1.0/pars[1])
            Netinfo[proj]['d_init'] = '%.1f' % x[0] 
            Netinfo[proj]['d_final'] = '%.1f' % x[-1]
            Netinfo[proj]['conn_prob_12.5um'] = '%.5f' % prob2D[0]
            Netinfo[proj]['conn_prob_25um'] = '%.5f' % prob2D[1]
            Netinfo[proj]['conn_prob_50um'] = '%.5f' % prob2D[2]
            Netinfo[proj]['conn_prob_75um'] = '%.5f' % prob2D[3]
            Netinfo[proj]['conn_prob_100um'] = '%.5f' % prob2D[4]
            Netinfo[proj]['conn_prob_125um'] = '%.5f' % prob2D[5]
            Netinfo[proj]['conn_prob_150um'] = '%.5f' % prob2D[6]
            Netinfo[proj]['conn_prob_175um'] = '%.5f' % prob2D[7]
            Netinfo[proj]['conn_prob_200um'] = '%.5f' % prob2D[8]
            Netinfo[proj]['conn_prob_225um'] = '%.5f' % prob2D[9]
            Netinfo[proj]['conn_prob_250um'] = '%.5f' % prob2D[10]
            Netinfo[proj]['conn_prob_275um'] = '%.5f' % prob2D[11]
            Netinfo[proj]['conn_prob_300um'] = '%.5f' % prob2D[12]
            Netinfo[proj]['conn_prob_325um'] = '%.5f' % prob2D[13]
            Netinfo[proj]['conn_prob_350um'] = '%.5f' % prob2D[14]
            Netinfo[proj]['conn_prob_375um'] = '%.5f' % prob2D[15]
            Netinfo[proj]['A0g'] = '%.5f' % popt[0]
            Netinfo[proj]['x0g'] = '%.2f' % popt[1]
            Netinfo[proj]['shapeg'] = '%.2f' % popt[2]
            
         
            # plot probs
            if plotdata:
                fontsiz=18
                figSize = (12,5)
                fig = plt.figure(figsize=figSize)  # Open a new figure
#                 fig.suptitle('%s -> A0=%.3f, l=%.1f, %0.1f<d<%d, NC=%.1f, p=%.4f, cov=%.2fe-6' % (proj,pars[0],1.0/pars[1],x[0],x[-1],nn/mm,pmat/mm,cov1), fontsize=15, fontweight='bold')
                if x[0] == 12.5:
                    print('%s %d %d %.3f %.1f %.1f %.0f %d %.3f %.1f %.1f %.1f' % 
                          (proj,n,m,pars[0],(1.0/pars[1]),(nn/mm),0.0,x[-1],popt[0],popt[1],popt[2],cov1-cov2))
                    fig.suptitle('%s (%.1f conn) -> Exp(%.3f, %.1f, %.0f<d<%d), Gauss(%.3f, %.1f, %.1f)' % (proj,nn/mm,pars[0],1.0/pars[1],0.0,x[-1],popt[0],popt[1],popt[2]), fontsize=15, fontweight='bold')
                else:
                    print('%s %d %d %.3f %.1f %.1f %d %d %.3f %.1f %.1f %.1f' % 
                          (proj,n,m,pars[0],(1.0/pars[1]),(nn/mm),x[0],x[-1],popt[0],popt[1],popt[2],cov1-cov2))    
                    fig.suptitle('%s (%.1f conn) -> Exp(%.3f, %.1f, %.0f<d<%d), Gauss(%.3f, %.1f, %.1f)' % (proj,nn/mm,pars[0],1.0/pars[1],x[0],x[-1],popt[0],popt[1],popt[2]), fontsize=15, fontweight='bold')
                xgg = np.linspace(0, endsexp, 50)
                plt.subplot(1, 1, 1)
                plt.ylabel('Prob2D', fontsize=fontsiz)
                plt.plot(d2Dfulldata, prob2Dfulldata, 'c.', label='7mcs')
                plt.plot(d2D, prob2D, 'ko', label='mean')
                plt.plot(x, y, 'bo', label='used') 
                plt.plot(xx, yy, 'b-', label='fit-e') 
                plt.plot(xi, yi, 'b-') 
                plt.plot(xgg, Gauss(xgg, *popt), 'r-', label='fit-g')
                plt.xlabel('distance (um)', fontsize=fontsiz)
                plt.xlim(0, 400)
                # ~ plt.ylim(ylim)
                plt.grid(False)
                plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.0))
                # plt.ion()
                plt.tight_layout()
                if nn/mm >= 1000:
                    if mtype_map[n] in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_ge1000/EE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] in Epops and mtype_map[m] not in Epops:
                        plt.savefig(rootFolder+'Figures_ge1000/EI/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] not in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_ge1000/IE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    else:
                        plt.savefig(rootFolder+'Figures_ge1000/II/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                elif nn/mm < 100:
                    if mtype_map[n] in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_l100/EE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] in Epops and mtype_map[m] not in Epops:
                        plt.savefig(rootFolder+'Figures_l100/EI/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] not in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_l100/IE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    else:
                        plt.savefig(rootFolder+'Figures_l100/II/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                else:
                    if mtype_map[n] in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_ge100_l1000/EE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] in Epops and mtype_map[m] not in Epops:
                        plt.savefig(rootFolder+'Figures_ge100_l1000/EI/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    elif mtype_map[n] not in Epops and mtype_map[m] in Epops:
                        plt.savefig(rootFolder+'Figures_ge100_l1000/IE/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                    else:
                        plt.savefig(rootFolder+'Figures_ge100_l1000/II/prob_dist2D_%s.png' % proj, facecolor = 'white' , dpi=300)
                plt.close(fig)

L1_DAC:L1_DAC 1 1 0.075 71.9 32.7 50 350 0.035 17.7 80.1 1.5
L1_DAC:L1_NGC-DA 1 2 0.056 62.4 24.4 50 325 0.051 -83.4 107.8 -257.1
L1_DAC:L1_NGC-SA 1 3 0.128 46.8 20.9 75 275 0.033 16.3 69.6 198.1
L1_DAC:L1_HAC 1 4 0.105 56.4 39.1 125 375 0.058 -95.4 120.9 -74.2
L1_DAC:L1_DLAC 1 5 0.110 64.9 13.7 125 375 0.039 -60.9 133.3 333.3
L1_DAC:L1_SLAC 1 6 0.078 63.0 17.0 75 325 0.023 56.0 61.1 14.0
L1_DAC:L23_PC 1 7 0.072 74.0 3208.3 75 375 0.044 -45.4 114.3 4.6
L1_DAC:L23_MC 1 8 0.103 75.7 212.7 125 375 0.034 -22.6 129.5 129.7
L1_DAC:L23_BTC 1 9 0.122 59.1 44.7 150 325 0.065 -178.2 161.6 608.5
L1_DAC:L23_DBC 1 10 0.042 56.7 32.6 125 325 0.018 -34.3 96.0 84.4
L1_DAC:L23_BP 1 11 0.000 -37.7 1.7 150 300 0.008 37.1 33.1 -0.1
L1_DAC:L23_NGC 1 12 0.067 48.0 11.7 100 350 0.000 0.0 1.0 642.3
L1_DAC:L23_LBC 1 13 0.049 66.0 137.7 75 350 0.023 -14.3 97.1 1.1
L1_DAC:L23_NBC 1 14 0.047 75.5 95.4 100 375 0.028 -55.3 123.4 -2.1
L1_DAC:L23_SBC 1 15 0.046 71.4 50.3 125 375 0.000 0.0 1.0 35.0
L1_DAC:L23_ChC 1 16

L1_DLAC:L5_STPC 5 32 0.101 63.0 53.3 125 375 0.021 27.2 97.7 263.9
L1_DLAC:L6_UTPC 5 44 0.001 132.1 11.4 125 350 0.000 137.4 100.9 0.0
L1_SLAC:L1_DAC 6 1 0.286 31.9 15.9 75 300 0.043 32.3 44.7 334.4
L1_SLAC:L1_NGC-DA 6 2 0.046 54.4 10.6 0 200 0.092 -105.8 89.0 -6077.2
L1_SLAC:L1_NGC-SA 6 3 0.063 46.3 8.7 0 200 0.068 -64.7 77.6 -293.2
L1_SLAC:L1_HAC 6 4 0.059 54.1 17.0 0 175 0.058 -53.7 80.0 -257.9
L1_SLAC:L1_DLAC 6 5 0.203 35.3 5.0 125 225 0.115 -108.4 92.9 8887.3
L1_SLAC:L1_SLAC 6 6 0.064 51.8 7.1 50 175 0.021 55.3 37.2 74.3
L1_SLAC:L23_PC 6 7 0.059 51.9 1076.9 50 350 0.033 -19.6 75.1 10.6
L1_SLAC:L23_MC 6 8 0.087 51.3 70.3 100 275 0.029 -32.7 97.5 106.6
L1_SLAC:L23_BTC 6 9 0.023 63.2 13.7 125 275 0.000 0.0 1.0 41.7
L1_SLAC:L23_DBC 6 10 0.016 52.8 8.9 0 200 0.016 -45.5 74.0 -42.3
L1_SLAC:L23_NGC 6 12 0.035 44.0 4.0 50 175 0.011 45.5 37.2 51.6
L1_SLAC:L23_LBC 6 13 0.041 41.4 36.7 50 275 0.017 3.6 56.3 13.9
L1_SLAC:L23_NBC 6 14 0.125 32.1 26.3 125 250 0.031 -46.2 73.4 6963.0
L1_SLAC:L23

L23_BTC:L1_NGC-DA 9 2 0.029 54.7 17.3 0 200 0.017 16.7 49.9 2.7
L23_BTC:L1_NGC-SA 9 3 0.027 57.8 11.6 25 225 0.016 29.2 43.5 14.1
L23_BTC:L1_HAC 9 4 0.056 39.0 22.4 50 300 0.020 9.1 52.1 6.2
L23_BTC:L1_DLAC 9 5 0.046 54.6 9.6 25 225 0.231 -238.0 129.1 -59425.2
L23_BTC:L1_SLAC 9 6 0.052 44.3 12.0 75 275 0.019 -3.6 68.7 6.8
L23_BTC:L23_PC 9 7 0.137 47.5 5333.7 50 350 0.075 -20.0 70.8 55.2
L23_BTC:L23_MC 9 8 0.116 63.9 393.1 75 325 0.067 -46.7 105.0 80.1
L23_BTC:L23_BTC 9 9 0.189 45.7 112.6 75 275 0.055 17.4 62.5 164.4
L23_BTC:L23_DBC 9 10 0.075 46.0 93.1 0 350 0.435 -268.4 133.5 -61742.7
L23_BTC:L23_BP 9 11 0.086 30.5 5.9 75 300 0.199 -245.7 121.4 -568846.3
L23_BTC:L23_NGC 9 12 0.100 45.5 36.7 50 275 0.062 -35.0 73.8 -102.6
L23_BTC:L23_LBC 9 13 0.169 47.1 460.3 75 325 0.063 -10.0 75.5 45.5
L23_BTC:L23_NBC 9 14 0.156 51.5 298.0 75 325 0.071 -28.3 86.5 54.1
L23_BTC:L23_SBC 9 15 0.072 62.9 134.3 0 275 0.241 -236.9 143.8 -7669.5
L23_BTC:L23_ChC 9 16 0.121 48.9 57.6 0 350 1.374 -351.3 152.7 -

L23_NGC:L23_PC 12 7 0.124 46.2 2309.9 75 375 0.042 -0.2 71.2 91.0
L23_NGC:L23_MC 12 8 0.095 57.4 153.4 50 300 0.052 -17.9 81.4 2.6
L23_NGC:L23_BTC 12 9 0.055 65.5 33.7 0 200 0.039 -4.5 69.3 5.2
L23_NGC:L23_DBC 12 10 0.037 53.5 27.6 25 200 0.023 10.7 51.3 13.9
L23_NGC:L23_NGC 12 12 0.068 45.6 12.1 50 200 0.021 45.3 39.6 44.8
L23_NGC:L23_LBC 12 13 0.086 51.1 157.3 50 300 0.045 -12.2 72.2 35.4
L23_NGC:L23_NBC 12 14 0.092 52.6 107.0 50 250 0.063 -42.1 83.5 26.9
L23_NGC:L23_SBC 12 15 0.108 41.7 47.0 75 275 0.029 11.5 62.4 3.1
L23_NGC:L23_ChC 12 16 0.071 52.4 18.0 25 175 0.046 2.0 54.6 22.5
L23_NGC:L4_PC 12 17 0.052 64.0 836.0 50 325 0.032 -24.8 89.6 7.3
L23_NGC:L4_SP 12 18 0.029 59.8 171.9 50 275 0.018 -25.5 85.2 3.8
L23_NGC:L4_SS 12 19 0.038 52.2 54.3 100 275 0.019 -96.2 119.5 -12.2
L23_NGC:L4_MC 12 20 0.019 51.3 11.6 0 225 0.000 0.0 1.0 1.2
L23_NGC:L4_LBC 12 25 0.030 49.1 11.4 125 325 0.000 0.0 1.0 128.0
L23_NGC:L4_NBC 12 26 0.064 38.3 7.0 125 225 0.520 -648.1 229.5 -35462207.7
L23_NGC:L4

L23_SBC:L23_DBC 15 10 0.075 39.0 93.6 50 300 0.028 9.4 51.4 54.6
L23_SBC:L23_BP 15 11 0.022 54.0 6.9 0 150 0.016 8.3 46.5 4.8
L23_SBC:L23_NGC 15 12 0.099 40.9 39.3 75 250 0.027 5.9 64.7 245.5
L23_SBC:L23_LBC 15 13 0.105 55.4 640.9 50 375 0.056 -8.6 74.4 49.0
L23_SBC:L23_NBC 15 14 0.094 54.6 334.7 50 300 0.050 -9.8 74.0 46.7
L23_SBC:L23_SBC 15 15 0.110 49.3 189.9 75 350 0.037 7.5 71.1 59.1
L23_SBC:L23_ChC 15 16 0.090 52.4 66.9 50 250 0.052 -22.1 76.5 60.0
L23_SBC:L4_PC 15 17 0.073 61.1 3187.7 50 375 0.049 -38.8 91.6 8.1
L23_SBC:L4_SP 15 18 0.046 57.6 745.7 50 375 0.028 -25.5 82.7 7.9
L23_SBC:L4_SS 15 19 0.042 59.2 249.1 75 375 0.018 -12.1 89.0 4.8
L23_SBC:L4_MC 15 20 0.035 56.4 57.6 75 275 0.020 -40.8 93.6 -5.1
L23_SBC:L4_BTC 15 21 0.045 40.4 7.3 50 250 0.015 26.8 43.7 27.2
L23_SBC:L4_DBC 15 22 0.062 35.8 12.0 75 300 0.009 47.6 41.5 86.5
L23_SBC:L4_LBC 15 25 0.030 54.7 51.4 50 250 0.031 -93.7 102.0 -53.2
L23_SBC:L4_NBC 15 26 0.120 36.5 43.1 125 275 0.017 -14.2 78.9 234.4
L23_SBC:L4_SBC 

...

L6_UTPC:L6_ChC 44 55 0.096 45.5 185.4 0 325 25.108 -620.6 183.7 -7180917875.8
L6_IPC:L23_PC 45 7 0.000 901.1 1827.9 175 375 0.000 248.7 155.8 0.0
L6_IPC:L23_MC 45 8 0.000 373.2 59.3 175 375 0.000 221.8 134.7 0.0
L6_IPC:L23_BTC 45 9 0.000 350.1 20.6 175 375 0.000 218.7 122.4 0.0
L6_IPC:L23_LBC 45 13 0.000 1887.0 66.7 175 375 0.000 264.7 157.6 0.0
L6_IPC:L23_NBC 45 14 0.000 648.7 40.4 175 375 0.000 243.5 130.4 0.0
L6_IPC:L23_SBC 45 15 0.000 -5806.7 17.4 175 375 0.000 336.9 320.5 0.0
L6_IPC:L23_ChC 45 16 0.000 -508.9 6.9 175 375 0.000 347.2 185.7 0.0
L6_IPC:L4_PC 45 17 0.001 266.6 4248.7 175 375 0.001 168.9 163.0 0.0
L6_IPC:L4_SP 45 18 0.001 241.2 1247.3 175 375 0.000 187.1 141.3 0.0
L6_IPC:L4_SS 45 19 0.001 292.8 640.6 175 375 0.001 163.5 186.6 0.0
L6_IPC:L4_MC 45 20 0.001 206.4 90.3 175 375 0.000 167.4 142.0 0.0
L6_IPC:L4_BTC 45 21 0.001 134.5 8.7 150 375 0.000 89.9 153.9 0.1
L6_IPC:L4_DBC 45 22 0.000 316.2 11.1 100 375 0.000 147.4 183.4 0.0
L6_IPC:L4_LBC 45 25 0.000 315.9 91.4 125 375 

L6_MC:L5_BP 47 36 0.050 46.8 26.6 125 325 0.000 0.0 1.0 53.3
L6_MC:L5_NGC 47 37 0.024 43.0 3.4 25 225 0.015 -4.0 49.3 -5.2
L6_MC:L5_LBC 47 38 0.051 56.7 322.3 50 375 0.051 -92.8 104.2 -87.8
L6_MC:L5_NBC 47 39 0.100 43.1 292.0 100 375 0.040 -68.8 98.2 9.2
L6_MC:L5_SBC 47 40 0.054 47.7 28.6 50 350 0.026 -7.5 66.6 13.6
L6_MC:L5_ChC 47 41 0.090 42.2 29.0 75 300 0.042 -46.9 82.8 29.5
L6_MC:L6_TPC_L1 47 42 0.085 50.2 3492.3 50 375 0.059 -44.9 82.2 -66.6
L6_MC:L6_TPC_L4 47 43 0.093 49.4 3281.7 50 375 0.072 -59.1 85.9 -189.3
L6_MC:L6_UTPC 47 44 0.088 55.2 4491.0 50 375 0.078 -74.4 97.0 -188.1
L6_MC:L6_IPC 47 45 0.066 54.8 6841.7 50 375 0.143 -187.7 127.4 -13480.0
L6_MC:L6_BPC 47 46 0.088 53.1 7693.3 50 375 0.077 -72.5 94.0 -212.6
L6_MC:L6_MC 47 47 0.074 54.6 738.0 50 375 0.088 -116.7 109.9 -2201.8
L6_MC:L6_BTC 47 48 0.070 55.8 114.6 50 350 0.172 -204.9 132.8 -19195.0
L6_MC:L6_DBC 47 49 0.054 40.5 29.0 50 375 0.030 -25.1 64.4 -11.2
L6_MC:L6_BP 47 50 0.028 62.3 72.0 0 375 0.000 0.0 1.0 0.3
L6_MC

/home/fernando/anaconda3/envs/netpynedev/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


L6_BP:L6_DBC 50 49 0.091 18.8 2.7 0 75 0.040 -17.6 38.6 -inf
L6_BP:L6_BP 50 50 0.055 38.7 8.0 125 225 0.000 0.0 1.0 822.6
L6_BP:L6_LBC 50 52 0.106 22.8 70.3 0 250 0.127 -59.2 51.5 -1709.4
L6_BP:L6_NBC 50 53 0.092 25.5 30.7 0 225 0.094 -48.5 50.8 -773.3
L6_BP:L6_SBC 50 54 0.062 24.4 6.1 0 175 0.040 -18.5 39.7 -52.2
L6_NGC:L5_TTPC1 51 29 0.011 59.9 44.0 50 300 0.025 -210.4 139.4 -367.3
L6_NGC:L5_TTPC2 51 30 0.012 37.1 20.4 0 200 0.016 -101.4 83.8 -58.5
L6_NGC:L5_STPC 51 32 0.024 40.7 5.4 50 150 0.007 39.0 37.9 44.2
L6_NGC:L5_MC 51 33 0.084 33.2 11.4 100 275 0.008 37.8 54.5 1488.0
L6_NGC:L5_LBC 51 38 0.050 34.0 4.1 100 300 0.085 -280.1 140.1 -60035.6
L6_NGC:L6_TPC_L1 51 42 0.110 47.6 190.6 0 200 0.073 7.7 44.4 51.4
L6_NGC:L6_TPC_L4 51 43 0.126 44.3 182.3 0 300 0.105 -22.2 56.1 -24.0
L6_NGC:L6_UTPC 51 44 0.270 33.5 273.0 50 275 0.099 -0.5 50.1 259.1
L6_NGC:L6_IPC 51 45 0.195 38.2 528.6 50 325 0.101 -22.7 62.6 4.8
L6_NGC:L6_BPC 51 46 0.247 34.8 500.4 50 350 0.100 -7.2 53.9 73.7
L6_NGC:L6_MC

L6_ChC:L6_IPC 55 45 0.026 75.8 185.9 125 375 0.000 0.0 1.0 14.5
L6_ChC:L6_BPC 55 46 0.017 71.7 107.9 125 350 0.000 0.0 1.0 1.0


In [3]:
# print(Netinfo)
with open(rootFolder+'Netconnections_mean_exp_gauss.json', 'w') as outfile:
    json.dump(Netinfo, outfile)

In [6]:
Netinfo['L23_PC:L23_PC']

{'conn_total': '369455.6',
 'conn_prob_full': '0.01070',
 'A0': '0.10239',
 'shape': '62.30',
 'd_init': '50.0',
 'd_final': '375.0',
 'conn_prob_12.5um': '0.06589',
 'conn_prob_25um': '0.05691',
 'conn_prob_50um': '0.04476',
 'conn_prob_75um': '0.03209',
 'conn_prob_100um': '0.02176',
 'conn_prob_125um': '0.01405',
 'conn_prob_150um': '0.00869',
 'conn_prob_175um': '0.00524',
 'conn_prob_200um': '0.00317',
 'conn_prob_225um': '0.00197',
 'conn_prob_250um': '0.00126',
 'conn_prob_275um': '0.00085',
 'conn_prob_300um': '0.00060',
 'conn_prob_325um': '0.00045',
 'conn_prob_350um': '0.00037',
 'conn_prob_375um': '0.00030',
 'A0g': '0.09421',
 'x0g': '-81.17',
 'shapeg': '106.39'}